In [1]:
%load_ext autoreload
%autoreload 2

## Links/tutorials

* http://karpathy.github.io/2015/05/21/rnn-effectiveness/
* http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
* http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
# load ascii text and covert to lowercase
filename = "/home/jvdzwaan/data/alice.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
print raw_text[:100]

alice’s adventures in wonderland

lewis carroll

the millennium fulcrum edition 3.0

chapter


In [5]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
print chars

['\n', '\r', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x98', '\x99', '\x9c', '\x9d', '\xe2']


In [8]:
print char_to_int

{'\x80': 44, '\n': 0, '\r': 1, '\x99': 46, '\x98': 45, '\x9d': 48, '\x9c': 47, '!': 3, ' ': 2, ')': 5, '(': 4, '*': 6, '-': 8, ',': 7, '.': 9, '0': 10, '3': 11, ';': 13, ':': 12, '?': 14, '\xe2': 49, '[': 15, ']': 16, '_': 17, 'a': 18, 'c': 20, 'b': 19, 'e': 22, 'd': 21, 'g': 24, 'f': 23, 'i': 26, 'h': 25, 'k': 28, 'j': 27, 'm': 30, 'l': 29, 'o': 32, 'n': 31, 'q': 34, 'p': 33, 's': 36, 'r': 35, 'u': 38, 't': 37, 'w': 40, 'v': 39, 'y': 42, 'x': 41, 'z': 43}


In [9]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  153739
Total Vocab:  50


In [10]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  153639


In [14]:
print seq_in
print '---'
print repr(seq_out)
print len(seq_in)

simple joys,
remembering her own child-life, and the happy summer days.

              the end

---
'\n'
100


In [15]:
for i in range(0, n_chars - seq_length, 1):
    print i

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3209
3210
3211
3212
3213
3214
3215
3216
3217
3218
3219
3220
3221
3222
3223
3224
3225
3226
3227
3228
3229
3230
3231
3232
3233
3234
3235
3236
3237
3238
3239
3240
3241
3242
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353


7420
7421
7422
7423
7424
7425
7426
7427
7428
7429
7430
7431
7432
7433
7434
7435
7436
7437
7438
7439
7440
7441
7442
7443
7444
7445
7446
7447
7448
7449
7450
7451
7452
7453
7454
7455
7456
7457
7458
7459
7460
7461
7462
7463
7464
7465
7466
7467
7468
7469
7470
7471
7472
7473
7474
7475
7476
7477
7478
7479
7480
7481
7482
7483
7484
7485
7486
7487
7488
7489
7490
7491
7492
7493
7494
7495
7496
7497
7498
7499
7500
7501
7502
7503
7504
7505
7506
7507
7508
7509
7510
7511
7512
7513
7514
7515
7516
7517
7518
7519
7520
7521
7522
7523
7524
7525
7526
7527
7528
7529
7530
7531
7532
7533
7534
7535
7536
7537
7538
7539
7540
7541
7542
7543
7544
7545
7546
7547
7548
7549
7550
7551
7552
7553
7554
7555
7556
7557
7558
7559
7560
7561
7562
7563
7564
7565
7566
7567
7568
7569
7570
7571
7572
7573
7574
7575
7576
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7587
7588
7589
7590
7591
7592
7593
7594
7595
7596
7597
7598
7599
7600
7601
7602
7603
7604
7605
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
7618
7619


11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11464
11465
11466
11467
11468
11469
11470
11471
11472
11473
11474
11475
11476
11477
11478
11479
11480
11481
11482
11483
11484
11485
11486
11487
11488
11489
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11505
11506
11507
11508
11509
11510
11511
11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
11579
11580
11581
11582
11583
11584
11585
11586
11587
11588
11589
11590
11591
11592
11593
11594
11595
11596
11597
11598
11599
11600
11601
11602
11603
11604
11605
11606
11607
11608
11609
11610
11611
11612
11613
11614
11615
11616
11617
1161

15911
15912
15913
15914
15915
15916
15917
15918
15919
15920
15921
15922
15923
15924
15925
15926
15927
15928
15929
15930
15931
15932
15933
15934
15935
15936
15937
15938
15939
15940
15941
15942
15943
15944
15945
15946
15947
15948
15949
15950
15951
15952
15953
15954
15955
15956
15957
15958
15959
15960
15961
15962
15963
15964
15965
15966
15967
15968
15969
15970
15971
15972
15973
15974
15975
15976
15977
15978
15979
15980
15981
15982
15983
15984
15985
15986
15987
15988
15989
15990
15991
15992
15993
15994
15995
15996
15997
15998
15999
16000
16001
16002
16003
16004
16005
16006
16007
16008
16009
16010
16011
16012
16013
16014
16015
16016
16017
16018
16019
16020
16021
16022
16023
16024
16025
16026
16027
16028
16029
16030
16031
16032
16033
16034
16035
16036
16037
16038
16039
16040
16041
16042
16043
16044
16045
16046
16047
16048
16049
16050
16051
16052
16053
16054
16055
16056
16057
16058
16059
16060
16061
16062
16063
16064
16065
16066
16067
16068
16069
16070
16071
16072
16073
16074
16075
16076
1607

...

135772
135773
135774
135775
135776
135777
135778
135779
135780
135781
135782
135783
135784
135785
135786
135787
135788
135789
135790
135791
135792
135793
135794
135795
135796
135797
135798
135799
135800
135801
135802
135803
135804
135805
135806
135807
135808
135809
135810
135811
135812
135813
135814
135815
135816
135817
135818
135819
135820
135821
135822
135823
135824
135825
135826
135827
135828
135829
135830
135831
135832
135833
135834
135835
135836
135837
135838
135839
135840
135841
135842
135843
135844
135845
135846
135847
135848
135849
135850
135851
135852
135853
135854
135855
135856
135857
135858
135859
135860
135861
135862
135863
135864
135865
135866
135867
135868
135869
135870
135871
135872
135873
135874
135875
135876
135877
135878
135879
135880
135881
135882
135883
135884
135885
135886
135887
135888
135889
135890
135891
135892
135893
135894
135895
135896
135897
135898
135899
135900
135901
135902
135903
135904
135905
135906
135907
135908
135909
135910
135911
135912
135913
135914

139870
139871
139872
139873
139874
139875
139876
139877
139878
139879
139880
139881
139882
139883
139884
139885
139886
139887
139888
139889
139890
139891
139892
139893
139894
139895
139896
139897
139898
139899
139900
139901
139902
139903
139904
139905
139906
139907
139908
139909
139910
139911
139912
139913
139914
139915
139916
139917
139918
139919
139920
139921
139922
139923
139924
139925
139926
139927
139928
139929
139930
139931
139932
139933
139934
139935
139936
139937
139938
139939
139940
139941
139942
139943
139944
139945
139946
139947
139948
139949
139950
139951
139952
139953
139954
139955
139956
139957
139958
139959
139960
139961
139962
139963
139964
139965
139966
139967
139968
139969
139970
139971
139972
139973
139974
139975
139976
139977
139978
139979
139980
139981
139982
139983
139984
139985
139986
139987
139988
139989
139990
139991
139992
139993
139994
139995
139996
139997
139998
139999
140000
140001
140002
140003
140004
140005
140006
140007
140008
140009
140010
140011
140012

143862
143863
143864
143865
143866
143867
143868
143869
143870
143871
143872
143873
143874
143875
143876
143877
143878
143879
143880
143881
143882
143883
143884
143885
143886
143887
143888
143889
143890
143891
143892
143893
143894
143895
143896
143897
143898
143899
143900
143901
143902
143903
143904
143905
143906
143907
143908
143909
143910
143911
143912
143913
143914
143915
143916
143917
143918
143919
143920
143921
143922
143923
143924
143925
143926
143927
143928
143929
143930
143931
143932
143933
143934
143935
143936
143937
143938
143939
143940
143941
143942
143943
143944
143945
143946
143947
143948
143949
143950
143951
143952
143953
143954
143955
143956
143957
143958
143959
143960
143961
143962
143963
143964
143965
143966
143967
143968
143969
143970
143971
143972
143973
143974
143975
143976
143977
143978
143979
143980
143981
143982
143983
143984
143985
143986
143987
143988
143989
143990
143991
143992
143993
143994
143995
143996
143997
143998
143999
144000
144001
144002
144003
144004

147862
147863
147864
147865
147866
147867
147868
147869
147870
147871
147872
147873
147874
147875
147876
147877
147878
147879
147880
147881
147882
147883
147884
147885
147886
147887
147888
147889
147890
147891
147892
147893
147894
147895
147896
147897
147898
147899
147900
147901
147902
147903
147904
147905
147906
147907
147908
147909
147910
147911
147912
147913
147914
147915
147916
147917
147918
147919
147920
147921
147922
147923
147924
147925
147926
147927
147928
147929
147930
147931
147932
147933
147934
147935
147936
147937
147938
147939
147940
147941
147942
147943
147944
147945
147946
147947
147948
147949
147950
147951
147952
147953
147954
147955
147956
147957
147958
147959
147960
147961
147962
147963
147964
147965
147966
147967
147968
147969
147970
147971
147972
147973
147974
147975
147976
147977
147978
147979
147980
147981
147982
147983
147984
147985
147986
147987
147988
147989
147990
147991
147992
147993
147994
147995
147996
147997
147998
147999
148000
148001
148002
148003
148004

151899
151900
151901
151902
151903
151904
151905
151906
151907
151908
151909
151910
151911
151912
151913
151914
151915
151916
151917
151918
151919
151920
151921
151922
151923
151924
151925
151926
151927
151928
151929
151930
151931
151932
151933
151934
151935
151936
151937
151938
151939
151940
151941
151942
151943
151944
151945
151946
151947
151948
151949
151950
151951
151952
151953
151954
151955
151956
151957
151958
151959
151960
151961
151962
151963
151964
151965
151966
151967
151968
151969
151970
151971
151972
151973
151974
151975
151976
151977
151978
151979
151980
151981
151982
151983
151984
151985
151986
151987
151988
151989
151990
151991
151992
151993
151994
151995
151996
151997
151998
151999
152000
152001
152002
152003
152004
152005
152006
152007
152008
152009
152010
152011
152012
152013
152014
152015
152016
152017
152018
152019
152020
152021
152022
152023
152024
152025
152026
152027
152028
152029
152030
152031
152032
152033
152034
152035
152036
152037
152038
152039
152040
152041

In [17]:
print dataX[0]
print dataY[0]

[18, 29, 26, 20, 22, 49, 44, 46, 36, 2, 18, 21, 39, 22, 31, 37, 38, 35, 22, 36, 2, 26, 31, 2, 40, 32, 31, 21, 22, 35, 29, 18, 31, 21, 1, 0, 1, 0, 29, 22, 40, 26, 36, 2, 20, 18, 35, 35, 32, 29, 29, 1, 0, 1, 0, 37, 25, 22, 2, 30, 26, 29, 29, 22, 31, 31, 26, 38, 30, 2, 23, 38, 29, 20, 35, 38, 30, 2, 22, 21, 26, 37, 26, 32, 31, 2, 11, 9, 10, 1, 0, 1, 0, 20, 25, 18, 33, 37, 22, 35]
2


In [19]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [20]:
print X

[[[ 0.36]
  [ 0.58]
  [ 0.52]
  ..., 
  [ 0.74]
  [ 0.44]
  [ 0.7 ]]

 [[ 0.58]
  [ 0.52]
  [ 0.4 ]
  ..., 
  [ 0.44]
  [ 0.7 ]
  [ 0.04]]

 [[ 0.52]
  [ 0.4 ]
  [ 0.44]
  ..., 
  [ 0.7 ]
  [ 0.04]
  [ 0.52]]

 ..., 
 [[ 0.7 ]
  [ 0.04]
  [ 0.72]
  ..., 
  [ 0.62]
  [ 0.42]
  [ 0.02]]

 [[ 0.04]
  [ 0.72]
  [ 0.52]
  ..., 
  [ 0.42]
  [ 0.02]
  [ 0.  ]]

 [[ 0.72]
  [ 0.52]
  [ 0.6 ]
  ..., 
  [ 0.02]
  [ 0.  ]
  [ 0.02]]]


In [21]:
print y

[[ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [22]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [23]:
# define the checkpoint
filepath="/home/jvdzwaan/data/tmp/lstm/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [24]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
153639/153639 [==============================] - 1355s - loss: 2.9780  
Epoch 2/20
153639/153639 [==============================] - 2243s - loss: 2.6174  
Epoch 3/20
  4480/153639 [..............................] - ETA: 2906s - loss: 2.5480

KeyboardInterrupt: 